In [1]:
import pandas as pd
journal_entries = pd.read_excel("Vinatics 2019 Journal Entries.xlsx")
chart_of_account = pd.read_excel("Vinatics Chart of Accounts.xlsx")
journal_entries.head()

,JE Number,GL Code,Period,Effective Date,Entry Date,Source,Source Group,Functional Amount,Preparer
0,210000003,60000001,2019-001,2019-01-18,2019-01-21,Invoice,Purchases,-302887.20000,Merrie Beaver
1,210000003,60000113,2019-001,2019-01-18,2019-01-21,Invoice,Purchases,250320.00000,Merrie Beaver
2,210000003,20001102,2019-001,2019-01-18,2019-01-21,Invoice,Purchases,52567.20000,Merrie Beaver
3,210000006,60000001,2019-001,2019-01-18,2019-01-21,Invoice,Purchases,-11708.75376,Merrie Beaver
4,210000006,60000112,2019-001,2019-01-18,2019-01-21,Invoice,Purchases,9676.65600,Merrie Beaver


In [2]:
journal_entries_copy = journal_entries.copy()
journal_entries_copy = journal_entries_copy[['GL Code','Source','Entry Date','Functional Amount']]
journal_entries_copy['Month'] = journal_entries_copy['Entry Date'].dt.month_name()
journal_entries_copy.head()

,GL Code,Source,Entry Date,Functional Amount,Month
0,60000001,Invoice,2019-01-21,-302887.20000,January
1,60000113,Invoice,2019-01-21,250320.00000,January
2,20001102,Invoice,2019-01-21,52567.20000,January
3,60000001,Invoice,2019-01-21,-11708.75376,January
4,60000112,Invoice,2019-01-21,9676.65600,January


In [3]:
chart_of_account.head()

,Account Number,Account Type,Class,Sub class,Account Name
0,20000050,Assets,Receivables,Trade receivables,Customer invoices
1,20000051,Assets,Receivables,Trade receivables,Credit notes
2,20000000,Assets,Receivables,Trade receivables,Receivables 1
3,20000021,Assets,Receivables,Trade receivables,Receivables 2
4,20000002,Assets,Receivables,Trade receivables,Receivables 3


In [5]:
journal_entries_copy = journal_entries_copy.merge(chart_of_account, how = 'outer', left_on='GL Code', right_on='Account Number').drop(['GL Code','Account Number','Entry Date','Account Name','Sub class'], axis = 1)
journal_entries_copy.head()

,Source,Functional Amount,Month,Account Type,Class
0,Invoice,-302887.20000,January,Liabilities,Payables
1,Invoice,-11708.75376,January,Liabilities,Payables
2,Invoice,-3758.66208,January,Liabilities,Payables
3,Invoice,-18907.95696,January,Liabilities,Payables
4,Invoice,-17307.84000,January,Liabilities,Payables


In [6]:
revenue = journal_entries_copy[journal_entries_copy['Account Type'] == 'Revenue']
expenses = journal_entries_copy[journal_entries_copy['Account Type'] == 'Expenses']

In [7]:
revenue_copy = revenue.copy()
revenue_copy = revenue_copy.dropna()
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September','October', 'November','December']

revenue_copy['Month'] = pd.Categorical(revenue_copy['Month'], categories=months, ordered=True)
revenue_copy = revenue_copy.sort_values(by = ['Month'])
revenue_copy['Functional Amount'] *= -1
revenue_copy = revenue_copy.pivot_table(columns='Month', index='Class', values='Functional Amount',aggfunc=sum)
revenue_copy = pd.DataFrame(revenue_copy.to_records())
revenue_copy = revenue_copy.fillna(0)
revenue_copy = revenue_copy.append(revenue_copy.sum(numeric_only=True), ignore_index=True)
revenue_copy = revenue_copy.append(pd.Series(), ignore_index=True)
revenue_copy['Total'] = revenue_copy.sum(axis = 1)
revenue_copy

,Class,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,Finance income,2.480334e+06,0.000000e+00,0.000000e+00,3.009353e+06,0.000000e+00,0.000000e+00,3.232596e+06,0.000000e+00,3.839096e+05,2.631735e+06,0.000000e+00,7.438080e+00,1.173794e+07
1,Foreign exchange gains,0.000000e+00,0.000000e+00,0.000000e+00,1.729354e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.729354e+02
2,Other income,1.644552e+07,6.042656e+05,4.986357e+06,3.512445e+05,3.386116e+05,2.139146e+06,9.699470e+06,4.639283e+05,1.278149e+06,2.890096e+05,3.379021e+05,2.130085e+06,3.906369e+07
3,Revenue from contract with customers,1.673393e+08,1.598645e+08,1.286180e+08,1.296631e+08,1.421999e+08,1.492517e+08,1.079696e+08,1.105862e+08,1.653530e+08,1.522117e+08,1.239356e+08,1.049408e+08,1.641933e+09
4,Sales of inventories,7.633568e+07,-6.150759e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.482809e+07
5,NaN,2.626008e+08,9.896117e+07,1.336044e+08,1.330239e+08,1.425385e+08,1.513909e+08,1.209017e+08,1.110501e+08,1.670151e+08,1.551325e+08,1.242735e+08,1.070709e+08,1.707563e+09
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00


In [8]:
expenses_copy = expenses.copy()
expenses_copy = expenses_copy.dropna()
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September','October', 'November','December']

expenses_copy['Month'] = pd.Categorical(expenses_copy['Month'], categories=months, ordered=True)
expenses_copy = expenses_copy.sort_values(by = ['Month'])
expenses_copy['Functional Amount'] *= -1
expenses_copy = expenses_copy.pivot_table(columns='Month', index='Class', values='Functional Amount',aggfunc=sum)
expenses_copy = pd.DataFrame(expenses_copy.to_records())
expenses_copy = expenses_copy.fillna(0)
expenses_copy = expenses_copy.append(expenses_copy.sum(numeric_only=True), ignore_index=True)
expenses_copy = expenses_copy.append(pd.Series(), ignore_index=True)
expenses_copy['Total'] = expenses_copy.sum(axis = 1)
expenses_copy

,Class,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,Cost of sales,-1.419600e+08,-2.522447e+07,-4.043989e+07,-6.385498e+07,-6.021648e+07,-3.848560e+07,-8.702008e+07,-5.466061e+07,-4.671700e+07,-7.588947e+07,-7.451821e+07,-7.757522e+07,-7.865620e+08
1,Depreciation and amortization,-3.394457e+06,-1.963824e+06,-2.678156e+06,-2.210459e+06,-2.499594e+06,-2.570115e+06,-2.443960e+06,-2.717375e+06,-2.813346e+06,-2.836460e+06,-2.846834e+06,-2.979257e+06,-3.195384e+07
2,Expected credit loss,-1.818721e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.818721e+07
3,Foreign exchange losses,-1.044192e+01,0.000000e+00,-1.165776e+02,-1.308816e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.514907e+02,-1.140744e+02,0.000000e+00,-9.234662e+02
4,Income tax expense,-7.025088e+07,0.000000e+00,6.984919e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.025088e+07
5,Interest expense,-4.156078e+05,-4.612256e+05,-4.608632e+05,-4.950556e+05,-4.790860e+05,-4.479798e+05,-3.967890e+05,-3.644581e+05,-3.523095e+05,-3.159616e+05,-2.798862e+05,-3.426494e+05,-4.811872e+06
6,Inventories,7.655735e+06,-9.543729e+07,-4.844148e+07,-4.023341e+07,-4.276404e+07,-4.243617e+07,-4.770727e+07,-2.190107e+07,-4.688375e+07,-4.624109e+07,-4.160792e+07,-3.186852e+07,-4.978663e+08
7,Other operating expenses,-6.824290e+05,-8.706154e+05,-8.000503e+05,-3.996206e+05,-1.086879e+06,-1.700705e+06,-3.688968e+06,-4.301222e+05,-7.927466e+05,-4.986384e+05,-1.581528e+06,-1.448586e+06,-1.398089e+07
8,Personnel costs,-2.452530e+05,-1.043835e+07,-1.170282e+07,-9.450553e+06,-1.015269e+07,-9.143047e+06,-1.116847e+07,-1.037130e+07,-9.011835e+06,-6.729752e+06,-8.695666e+06,-2.142246e+07,-1.185322e+08
9,Rentals,-2.174569e+06,-1.022588e+06,-8.646371e+05,-2.018146e+06,-7.825181e+05,-8.814109e+05,-6.366348e+05,-1.886003e+06,-8.305952e+05,-2.276754e+06,-1.147902e+06,-1.096111e+06,-1.561787e+07


In [9]:
net_income = revenue_copy.iloc[[-2]].append(expenses_copy.iloc[[-2]], ignore_index=True)
net_income = net_income.append(net_income.sum(numeric_only=True), ignore_index=True)
net_income = net_income.iloc[[-1]]
net_income = net_income.fillna('Net Income')
net_income

,Class,January,February,March,April,May,June,July,August,September,October,November,December,Total
2,Net Income,3.294616e+07,-3.645719e+07,2.821635e+07,1.436157e+07,2.455723e+07,5.572587e+07,-3.216050e+07,1.871916e+07,5.961349e+07,2.034378e+07,-6.404588e+06,-2.966188e+07,1.497994e+08


In [10]:
income_statement = revenue_copy.append(expenses_copy, ignore_index=True)
income_statement = income_statement.append(net_income, ignore_index=True)
income_statement

,Class,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,Finance income,2.480334e+06,0.000000e+00,0.000000e+00,3.009353e+06,0.000000e+00,0.000000e+00,3.232596e+06,0.000000e+00,3.839096e+05,2.631735e+06,0.000000e+00,7.438080e+00,1.173794e+07
1,Foreign exchange gains,0.000000e+00,0.000000e+00,0.000000e+00,1.729354e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.729354e+02
2,Other income,1.644552e+07,6.042656e+05,4.986357e+06,3.512445e+05,3.386116e+05,2.139146e+06,9.699470e+06,4.639283e+05,1.278149e+06,2.890096e+05,3.379021e+05,2.130085e+06,3.906369e+07
3,Revenue from contract with customers,1.673393e+08,1.598645e+08,1.286180e+08,1.296631e+08,1.421999e+08,1.492517e+08,1.079696e+08,1.105862e+08,1.653530e+08,1.522117e+08,1.239356e+08,1.049408e+08,1.641933e+09
4,Sales of inventories,7.633568e+07,-6.150759e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.482809e+07
5,NaN,2.626008e+08,9.896117e+07,1.336044e+08,1.330239e+08,1.425385e+08,1.513909e+08,1.209017e+08,1.110501e+08,1.670151e+08,1.551325e+08,1.242735e+08,1.070709e+08,1.707563e+09
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
7,Cost of sales,-1.419600e+08,-2.522447e+07,-4.043989e+07,-6.385498e+07,-6.021648e+07,-3.848560e+07,-8.702008e+07,-5.466061e+07,-4.671700e+07,-7.588947e+07,-7.451821e+07,-7.757522e+07,-7.865620e+08
8,Depreciation and amortization,-3.394457e+06,-1.963824e+06,-2.678156e+06,-2.210459e+06,-2.499594e+06,-2.570115e+06,-2.443960e+06,-2.717375e+06,-2.813346e+06,-2.836460e+06,-2.846834e+06,-2.979257e+06,-3.195384e+07
9,Expected credit loss,-1.818721e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.818721e+07


In [11]:
writer = pd.ExcelWriter('Monthly Income Statement.xlsx', engine = 'xlsxwriter')
income_statement.to_excel(writer, sheet_name='Income Statement', index=False, startrow=3)

workbook = writer.book
income_statement_sheet = writer.sheets['Income Statement']

#Excel Formats
money_format = workbook.add_format({'num_format': '_(* #,##0.00_);_(* (#,##0.00);_(* "-"??_);_(@_)','align': 'vjustify','valign': 'right'})
header_format = workbook.add_format({'bold': True,'valign': 'top','font_size':14})

#Format the income statement sheet
income_statement_sheet.set_zoom(90)
income_statement_sheet.write(0, 0, 'VINATICS', header_format)
income_statement_sheet.write(1, 0, 'MONTHLY INCOME STATEMENT', header_format)
income_statement_sheet.set_column('A:A', 35)
income_statement_sheet.set_column('B:N', 20, money_format)

writer.save()

import os
os.system('start "excel.exe" "Monthly Income Statement.xlsx"')

0